In [2]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

llm = AzureChatOpenAI(
    openai_api_key="4b94ea21e11a4d46b7b5c17a1ef08de4", 
    api_version='2023-09-01-preview', 
    azure_endpoint='https://egn-llm-hackathon.openai.azure.com/', 
    deployment_name="gpt-35-turbo"
    )
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x7065d8cc0790> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7065d8cc3f10> openai_api_key='4b94ea21e11a4d46b7b5c17a1ef08de4' openai_proxy='' azure_endpoint='https://egn-llm-hackathon.openai.azure.com/' deployment_name='gpt-35-turbo' openai_api_version='2023-09-01-preview' openai_api_type='azure'


In [3]:
message = HumanMessage(
    content="Translate this sentence from English to French. I love programming."
)
llm.invoke([message])

AIMessage(content="J'adore programmer.")

In [39]:
chain = prompt | llm 

In [42]:
res = chain.invoke({"input": "how can langsmith help with testing?"})

NameError: name 'pprint' is not defined

In [44]:
print(res.content)


Langsmith can greatly assist with testing by providing a powerful and flexible language generation platform. Here are some ways in which Langsmith can help with testing:

1. Test Case Generation: Langsmith can be used to automatically generate test cases for software applications. By leveraging its language generation capabilities, Langsmith can create a wide variety of test scenarios, covering different input values, edge cases, and boundary conditions. This can significantly increase test coverage and help uncover potential bugs or issues.

2. Test Data Generation: Langsmith can generate realistic and diverse test data for use in testing. It can create synthetic data that simulates real-world conditions, ensuring that the software under test is exposed to a wide range of inputs. This can be particularly useful for stress testing, performance testing, and security testing.

3. Test Script Automation: Langsmith can be integrated with testing frameworks and automation tools to automate 

In [4]:
from langchain_openai import AzureOpenAIEmbeddings
import os

embeddings_model = AzureOpenAIEmbeddings(
    openai_api_key="4b94ea21e11a4d46b7b5c17a1ef08de4", 
    api_version='2023-09-01-preview', 
    azure_endpoint='https://egn-llm-hackathon.openai.azure.com/')
print(embeddings_model)

client=<openai.resources.embeddings.Embeddings object at 0x7065d8ce60d0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7065d8cf8f50> model='text-embedding-ada-002' dimensions=None deployment=None openai_api_version='2023-09-01-preview' openai_api_base=None openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='4b94ea21e11a4d46b7b5c17a1ef08de4' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=16 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None azure_endpoint='https://egn-llm-hackathon.openai.azure.com/' azure_ad_token=None azure_ad_token_provider=None validate_base_url=True


In [5]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

In [19]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('queens_speeches/speeches', glob="**/*.txt")
docs = loader.load()

In [28]:
for i, doc in enumerate(docs):
    doc.metadata["year"] = int(doc.metadata["source"].split('/')[-1].split('.')[0])#.split('/')[-1].split['.'][0]


In [29]:
docs[0].metadata

{'source': 'queens_speeches/speeches/2003.txt', 'year': 2003}

In [30]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20, length_function = len, is_separator_regex = False)
#documents = text_splitter.split_documents(docs)
#input_documents = [text_splitter.create_documents([d.page_content]) for d in documents]

text_splitter = SemanticChunker(embeddings_model)
documents = text_splitter.split_documents(docs)


In [ ]:
#test_document = documents[0]
#test_document
#documents = text_splitter.create_documents([d.page_content for d in docs])

In [117]:
#input_documents = [y for x in input_documents for y in x]

In [31]:
vector = FAISS.from_documents(documents, embeddings_model)

In [42]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Besvar nedenstående spørgsmål, udelukkende baseret på denne kontekst:

<kontekst>
{context}
</kontekst>

Spørgsmål: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [43]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [47]:
response = retrieval_chain.invoke({"input": "Hvilket år blev Kronprins Frederiks første datter født?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

Kronprins Frederiks første datter blev født i år.
